In [13]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [14]:
url = "https://raw.githubusercontent.com/sayemuzzamansiam/Emotion-Detection-EN/main/dff.csv"

In [15]:
df=pd.read_csv(url)

In [16]:
# Check the data
print(df.head())


       emotion  emotion_num                                       cleaned_text
0      sadness           18                                          game hurt
1   admiration            0  sexuality shouldnt grouping category makes dif...
2         love           15                                    man love reddit
3    gratitude           13  right considering important document know damn...
4  disapproval            9  isnt big hes still quite popular ive heard thi...


In [17]:
# Preprocess the text data
df['cleaned_text'] = df['cleaned_text'].astype(str)  # Ensure all entries are strings
df['cleaned_text'].fillna('', inplace=True)  # Replace NaN values with empty strings

X = df['cleaned_text'].values
y = df['emotion_num'].values



In [18]:
# Tokenize the text
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(X)
word_index = tokenizer.word_index
X_sequences = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(X_sequences, maxlen=100, padding='post', truncating='post')



In [19]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

# Build the model


In [20]:
# Build the model
model = Sequential([
    Embedding(input_dim=5000, output_dim=64, input_length=100),
    Bidirectional(LSTM(64)),
    Dense(32, activation='relu'),
    Dense(len(np.unique(y)), activation='softmax')
])



In [21]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [22]:
# Print model summary
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 64)           320000    
                                                                 
 bidirectional_1 (Bidirecti  (None, 128)               66048     
 onal)                                                           
                                                                 
 dense_2 (Dense)             (None, 32)                4128      
                                                                 
 dense_3 (Dense)             (None, 21)                693       
                                                                 
Total params: 390869 (1.49 MB)
Trainable params: 390869 (1.49 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
# Train the model
history = model.fit(X_train, y_train, epochs=3, validation_data=(X_test, y_test), batch_size=32)

Epoch 1/3
4024/4024 [==============================] - 449s 110ms/step - loss: 2.3498 - accuracy: 0.3118 - val_loss: 2.1589 - val_accuracy: 0.3694
Epoch 2/3
4024/4024 [==============================] - 441s 110ms/step - loss: 2.0599 - accuracy: 0.3951 - val_loss: 2.0959 - val_accuracy: 0.3818
Epoch 3/3
4024/4024 [==============================] - 439s 109ms/step - loss: 1.9734 - accuracy: 0.4144 - val_loss: 2.0709 - val_accuracy: 0.3925


In [27]:
# Save the model
model.save('emotion_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [28]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')


1006/1006 [==============================] - 28s 27ms/step - loss: 2.0709 - accuracy: 0.3925
Loss: 2.0709173679351807
Accuracy: 0.39246878027915955


In [33]:
# Load the model
model = tf.keras.models.load_model('emotion_model.h5')

# Function to predict the emotion of a given sentence
def predict_emotion(sentence):
    # Preprocess the input sentence
    sentence = sentence.lower()  # Convert to lowercase
    sequence = tokenizer.texts_to_sequences([sentence])
    padded_sequence = pad_sequences(sequence, maxlen=100, padding='post', truncating='post')

    # Predict the emotion
    prediction = model.predict(padded_sequence)
    predicted_class = np.argmax(prediction, axis=1)[0]

    # Decode the predicted class into the corresponding emotion label
    emotions = {index: label for index, label in enumerate(df['emotion'].unique())}
    predicted_emotion = emotions[predicted_class]

    return predicted_emotion

# Example usage
sentence = "love you"
predicted_emotion = predict_emotion(sentence)
print(f"The predicted emotion for the sentence is: {predicted_emotion}")


1/1 [==============================] - 1s 784ms/step
The predicted emotion for the sentence is: disgust
